In [1]:
%cd /home/luke-padmore/Source/flow-matching-mnist

/home/luke-padmore/Source/flow-matching-mnist


In [2]:
import torchvision
import math
import torch
import matplotlib.pyplot as plt 
import torch.nn as nn 
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from models.unet import UNet, CondUNet
import os, sys
from models.ode_solvers import euler_solver, rk2_solver
from pathlib import Path 

# For system metrics
os.environ["MLFLOW_ENABLE_SYSTEM_METRICS_LOGGING"] = "true"
PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

batch_size = 64
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Pad(2,padding_mode='constant'),
    transforms.Normalize((0.5,), (0.5,))]
)
trainset = torchvision.datasets.MNIST(root = '/home/luke-padmore/Source/flow-matching-mnist/data',
                                      train=True,
                                      download=True,
                                      transform=transform)
trainloader = DataLoader(trainset,
                        batch_size=batch_size,
                        shuffle=True,
                        num_workers=4)
valset = torchvision.datasets.MNIST(root = '/home/luke-padmore/Source/flow-matching-mnist/data',
                                      train=False,
                                      download=True,
                                      transform=transform)
valloader = DataLoader(valset,
                        batch_size=batch_size,
                        shuffle=True,
                        num_workers=4)

/home/luke-padmore/miniconda3/envs/ml/lib/python3.12/site-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [3]:
from utils.FID.fid_model import FID_classifier
from utils.FID.fid_training import train

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = FID_classifier().to(device)
train(model,trainloader,valloader,100,1e-5,10,device)

/home/luke-padmore/miniconda3/envs/ml/lib/python3.12/site-packages/mlflow/tracking/_tracking_service/utils.py:178: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.
  return FileStore(store_uri, store_uri)
2026/01/02 00:40:39 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
2026-01-02 00:40:39,851 | INFO | fid_4e100465cf134ae4a316acd917b442f0 | Hyperparameters:
 num_epochs: 1
 lr: 1e-05
 conv_block_size: (1, 32, 64)
 n_classes: 10
 fid_embed: 128
Training:   0%|          | 0/1 [00:00<?, ?it/s]2026-01-02 00:40:50,185 | INFO | fid_4e100465cf134ae4a316acd917b442f0 | Epoch [1/1] | Loss: 1214.4000
2026-0